In [1]:
import numpy as np
import pandas as pd
import main

rigis = pd.read_csv(main.data_dir + "\\rigis.csv", delimiter=';')
del rigis['Пласт РИГИС']
del rigis['Пластпропласт']
del rigis['Литология']
del rigis['Коллектор']

rigis = rigis.rename(columns={'Скважина': 'well_name'})
rigis = rigis.rename(columns={'Абсолютная глубина': 'z_abs'})
rigis = rigis.rename(columns={'Абсолютная толщина': 'h_abs'})
rigis = rigis.rename(columns={'Глубина кровли пласта': 'z'})
rigis = rigis.rename(columns={'Толщина': 'h'})

rigis.head()

,well_name,z,h,z_abs,h_abs
0,334,2 748.60,1.0,2 530.41,1.0
1,334,2 749.60,1.2,2 531.41,1.2
2,334,2 750.80,0.4,2 532.61,0.4
3,358,2 677.60,0.6,2 532.72,0.6
4,334,2 751.20,0.4,2 533.01,0.4


In [2]:
rigis.replace(' ', '', regex=True, inplace=True)
rigis['z_abs'] = rigis['z_abs'].astype('float64')
rigis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3688 entries, 0 to 3687
Data columns (total 5 columns):
well_name    3688 non-null object
z            3688 non-null object
h            3688 non-null float64
z_abs        3688 non-null float64
h_abs        3687 non-null float64
dtypes: float64(3), object(2)
memory usage: 144.1+ KB


In [3]:
header_data = pd.read_csv(main.data_dir + "\Well_Header.csv", delimiter='\t')
header_data.head()
header_data = header_data.rename(columns={'WellName': 'well_name'})
del header_data['KB']
del header_data['TopDepth']
del header_data['BottomDepth']
del header_data['Symbol']
del header_data['Rig']
del header_data['Field']
header_data.head()

,well_name,X-Coord,Y-Coord
0,314_103ST2,354719.52,6470051.42
1,314_107,354693.18,6470164.36
2,314_121,354433.25,6468384.38
3,314_125PL,354717.25,6470061.16
4,314_134,354478.44,6468306.61


In [4]:
mlist = header_data['well_name'].values.tolist()
new_list = []
for x in mlist:
    new_list.append(x.replace('314_',''))
header_data['well_name']= new_list    
header_data.head()

,well_name,X-Coord,Y-Coord
0,103ST2,354719.52,6470051.42
1,107,354693.18,6470164.36
2,121,354433.25,6468384.38
3,125PL,354717.25,6470061.16
4,134,354478.44,6468306.61


In [5]:
rigis = pd.merge(rigis, header_data, on='well_name')
rigis.head()

,well_name,z,h,z_abs,h_abs,X-Coord,Y-Coord
0,334,2748.60,1.0,2530.41,1.0,351562.55,6464575.94
1,334,2749.60,1.2,2531.41,1.2,351562.55,6464575.94
2,334,2750.80,0.4,2532.61,0.4,351562.55,6464575.94
3,334,2751.20,0.4,2533.01,0.4,351562.55,6464575.94
4,334,2751.60,1.0,2533.41,1.0,351562.55,6464575.94


In [6]:
top = rigis.groupby(['well_name'])['z_abs'].min()

In [7]:
top_df =  top.to_frame().reset_index()
top_df.head()

,well_name,z_abs
0,100,2548.81
1,101,2556.23
2,102,2542.13
3,103,2541.53
4,103ST2,2566.04


In [8]:
top_df = pd.merge(top_df, header_data, on='well_name')
top_df.head()
top_df.shape

(231, 4)

In [9]:
top_df.to_csv('top_df.csv',sep =';')

In [ ]:
botom = rigis.groupby(['well_name'])['z_abs'].max()
botom_df =  botom.to_frame().reset_index()
botom_df.head()

In [ ]:
rigis_h = rigis[['well_name', 'z_abs', 'h_abs']]
botom_df =  pd.merge(botom_df, rigis_h, on=['well_name', 'z_abs'])
botom_df.head()

In [ ]:
botom_df.replace(' ', '', regex=True, inplace=True)
botom_df['z_abs'] = botom_df['z_abs'].astype('float64')
botom_df.info()

In [ ]:
botom_df['z_abs'] = botom_df['z_abs'] + botom_df['h_abs']

In [ ]:
del botom_df['h_abs']
botom_df.head()

In [ ]:
botom_df = pd.merge(botom_df, header_data, on='well_name')
botom_df.head()
# bottom_df.shape

In [ ]:
botom_df.to_csv('bottom_df.csv',sep =';')